In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor

In [34]:
df = pd.read_csv('income.csv')

In [35]:
df.head()

,Income,Age,Dependents,Occupation,City_Tier,Rent,Loan_Repayment,Insurance,Groceries,Transport,Eating_Out,Entertainment,Utilities,Healthcare,Education,Miscellaneous,Desired_Savings_Percentage,Desired_Savings,Disposable_Income,Potential_Savings_Groceries,Potential_Savings_Transport,Potential_Savings_Eating_Out,Potential_Savings_Entertainment,Potential_Savings_Utilities,Potential_Savings_Healthcare,Potential_Savings_Education,Potential_Savings_Miscellaneous
0,44637.249636,49,0,Self_Employed,Tier_1,13391.174891,0.000000,2206.490129,6658.768341,2636.970696,1651.801726,1536.184255,2911.792231,1546.914539,0.000000,831.525120,13.890948,6200.537192,11265.627707,1685.696222,328.895281,465.769172,195.151320,678.292859,67.682471,0.000000,85.735517
1,26858.596592,34,2,Retired,Tier_2,5371.719318,0.000000,869.522617,2818.444460,1543.018778,649.378103,1050.241738,1626.143346,1137.351325,1551.723167,564.235007,7.160376,1923.176434,9676.818733,540.306561,119.347139,141.866089,234.131168,286.668408,6.603212,56.306874,97.388606
2,50367.605084,35,1,Student,Tier_3,7555.140763,4612.103386,2201.800050,6313.222081,3221.396403,1513.814376,1723.306926,3368.457108,2178.515847,3160.026299,628.371220,13.997808,7050.360422,13891.450624,1466.073984,473.549752,410.857129,459.965256,488.383423,7.290892,106.653597,138.542422
3,101455.600247,21,0,Self_Employed,Tier_3,15218.340037,6809.441427,4889.418087,14690.149363,7106.130005,5040.249158,2858.194240,6128.550387,4571.118217,0.000000,2526.055712,16.455440,16694.965136,31617.953615,1875.932770,762.020789,1241.017448,320.190594,1389.815033,193.502754,0.000000,296.041183
4,24875.283548,52,4,Professional,Tier_2,4975.056710,3112.609398,635.907170,3034.329665,1276.155163,692.827225,660.186852,1092.689024,1169.095816,1445.215694,515.510300,7.533982,1874.099434,6265.700532,788.953124,68.160766,61.712505,187.173750,194.117130,47.294591,67.388120,96.557076


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 27 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Income                           20000 non-null  float64
 1   Age                              20000 non-null  int64  
 2   Dependents                       20000 non-null  int64  
 3   Occupation                       20000 non-null  object 
 4   City_Tier                        20000 non-null  object 
 5   Rent                             20000 non-null  float64
 6   Loan_Repayment                   20000 non-null  float64
 7   Insurance                        20000 non-null  float64
 8   Groceries                        20000 non-null  float64
 9   Transport                        20000 non-null  float64
 10  Eating_Out                       20000 non-null  float64
 11  Entertainment                    20000 non-null  float64
 12  Utilities         

In [4]:
#dtypes = df.select_dtypes(include=['object', 'int', 'float'])

In [37]:
# Correct way to round all float columns to 2 decimal places
for col in df.select_dtypes(include=['float']).columns:
    df[col] = df[col].round(2)

In [38]:
pd.set_option('display.max_columns', 50)

In [41]:
df.head()

,Income,Age,Dependents,Occupation,City_Tier,Rent,Loan_Repayment,Insurance,Groceries,Transport,Eating_Out,Entertainment,Utilities,Healthcare,Education,Miscellaneous,Desired_Savings_Percentage,Desired_Savings,Disposable_Income,Potential_Savings_Groceries,Potential_Savings_Transport,Potential_Savings_Eating_Out,Potential_Savings_Entertainment,Potential_Savings_Utilities,Potential_Savings_Healthcare,Potential_Savings_Education,Potential_Savings_Miscellaneous,Total_Spending
0,44637.25,49,0,Self_Employed,Tier_1,13391.17,0.00,2206.49,6658.77,2636.97,1651.80,1536.18,2911.79,1546.91,0.00,831.53,13.89,6200.54,11265.63,1685.70,328.90,465.77,195.15,678.29,67.68,0.00,85.74,54358.90
1,26858.60,34,2,Retired,Tier_2,5371.72,0.00,869.52,2818.44,1543.02,649.38,1050.24,1626.14,1137.35,1551.72,564.24,7.16,1923.18,9676.82,540.31,119.35,141.87,234.13,286.67,6.60,56.31,97.39,30271.56
2,50367.61,35,1,Student,Tier_3,7555.14,4612.10,2201.80,6313.22,3221.40,1513.81,1723.31,3368.46,2178.52,3160.03,628.37,14.00,7050.36,13891.45,1466.07,473.55,410.86,459.97,488.38,7.29,106.65,138.54,60983.28
3,101455.60,21,0,Self_Employed,Tier_3,15218.34,6809.44,4889.42,14690.15,7106.13,5040.25,2858.19,6128.55,4571.12,0.00,2526.06,16.46,16694.97,31617.95,1875.93,762.02,1241.02,320.19,1389.82,193.50,0.00,296.04,124245.55
4,24875.28,52,4,Professional,Tier_2,4975.06,3112.61,635.91,3034.33,1276.16,692.83,660.19,1092.69,1169.10,1445.22,515.51,7.53,1874.10,6265.70,788.95,68.16,61.71,187.17,194.12,47.29,67.39,96.56,28268.29


In [17]:
#df.columns

In [22]:
#df.groupby('Occupation')[['Income']].boxplot()

In [40]:
# First select all float columns
float_cols = df.select_dtypes(include=['float'])

# Then exclude 'Income' if it exists in the float columns
if 'Income' in float_cols.columns:
    float_cols = float_cols.drop(columns=['Income'])
    
# Sum across the remaining float columns
df['Total_Spending'] = float_cols.sum(axis=1)

In [44]:
df['Money_Left'] = df['Income'] - df['Total_Spending']

In [45]:
df.head()

,Income,Age,Dependents,Occupation,City_Tier,Rent,Loan_Repayment,Insurance,Groceries,Transport,Eating_Out,Entertainment,Utilities,Healthcare,Education,Miscellaneous,Desired_Savings_Percentage,Desired_Savings,Disposable_Income,Potential_Savings_Groceries,Potential_Savings_Transport,Potential_Savings_Eating_Out,Potential_Savings_Entertainment,Potential_Savings_Utilities,Potential_Savings_Healthcare,Potential_Savings_Education,Potential_Savings_Miscellaneous,Total_Spending,Money_Left
0,44637.25,49,0,Self_Employed,Tier_1,13391.17,0.00,2206.49,6658.77,2636.97,1651.80,1536.18,2911.79,1546.91,0.00,831.53,13.89,6200.54,11265.63,1685.70,328.90,465.77,195.15,678.29,67.68,0.00,85.74,54358.90,-9721.65
1,26858.60,34,2,Retired,Tier_2,5371.72,0.00,869.52,2818.44,1543.02,649.38,1050.24,1626.14,1137.35,1551.72,564.24,7.16,1923.18,9676.82,540.31,119.35,141.87,234.13,286.67,6.60,56.31,97.39,30271.56,-3412.96
2,50367.61,35,1,Student,Tier_3,7555.14,4612.10,2201.80,6313.22,3221.40,1513.81,1723.31,3368.46,2178.52,3160.03,628.37,14.00,7050.36,13891.45,1466.07,473.55,410.86,459.97,488.38,7.29,106.65,138.54,60983.28,-10615.67
3,101455.60,21,0,Self_Employed,Tier_3,15218.34,6809.44,4889.42,14690.15,7106.13,5040.25,2858.19,6128.55,4571.12,0.00,2526.06,16.46,16694.97,31617.95,1875.93,762.02,1241.02,320.19,1389.82,193.50,0.00,296.04,124245.55,-22789.95
4,24875.28,52,4,Professional,Tier_2,4975.06,3112.61,635.91,3034.33,1276.16,692.83,660.19,1092.69,1169.10,1445.22,515.51,7.53,1874.10,6265.70,788.95,68.16,61.71,187.17,194.12,47.29,67.39,96.56,28268.29,-3393.01
